<a href="https://colab.research.google.com/github/otanet/NLP_seminar_20201022/blob/main/mbart_summarization_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# mBARTを用いた要約生成の例
*   本ノートブックでは，日本語要約データにfinetuneされたmBARTモデルで要約を生成する例を示します．
*   huggingfaceの[transformers](https://github.com/huggingface/transformers)を使用します．



# 独自データを使ってfine-tuningする場合

transformersにサンプルコードがあるのでそちらを参考にしてください

1.   git clone https://github.com/huggingface/transformers.git
2.   %cd /path/transformers/examples/seq2seq
3.   [examplesの説明](https://github.com/huggingface/transformers/tree/master/examples)に従って必要なライブラリをインストール (important notes)
3.   train_mbart_cc25_enro.shを参考に日本語用に設定を一部変更します
4.   --src_lang en_XX --tgt_lang ro_ROの部分で言語を指定します．日本語の場合は--src_lang ja_XX --tgt_lang ja_XXとしてください
5.   --data_dir でデータの場所をしてください．データはtrain.source, train.target, val.source, val.target,test.source, test.targetの6種類が必要です
6.   --max_source_length で扱う入力長の最大値を設定できます．マシンによってはこの値を小さく設定してください．その他マシンによって適宜設定を変更してください．



# 学習済モデルを使って生成を試す場合

## 環境のセットアップ

In [ ]:
!pip install transformers==3.3.1

     |████████████████████████████████| 1.1MB 9.3MB/s 
     |████████████████████████████████| 3.0MB 33.0MB/s 
     |████████████████████████████████| 890kB 54.8MB/s 
     |████████████████████████████████| 1.1MB 47.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=636b1b55758ce0c2743e282305ec600176c1a1c6bbef61cee97b3b662aad1f45
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import MBartForConditionalGeneration, MBartTokenizer

## 学習済モデル・トークナイザ読み込み

発表中のデモで使用したlivedoor3行要約データ約10000件で学習済のモデルを下記においたので，学習済モデルで試す場合は[こちらのモデル](https://drive.google.com/file/d/1YMCVZd8sNvozegFbR73elDaeyNV7gY9U/view?usp=sharing)をダウンロード＆展開して指定してください




1.   上記のlivedoor_sample.tar.gzをダウンロード 
2.   tar zxvf livedoor_sample.tar.gz
3.   best_tfmrというディレクトリができる
4.   Colabの作業環境のどこかに配置．配置した場所を/path/model/best_tfmrとする




In [ ]:
model = MBartForConditionalGeneration.from_pretrained("/path/model/best_tfmr")
tokenizer = MBartTokenizer.from_pretrained("/path/model/best_tfmr")
model.cuda()

MBartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
      (layers): ModuleList(
        (0): EncoderLayer(
          (self_attn): Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=Tru

## ソーステキストの指定

*   ソーステキストを指定して，トークナイズ＆id化する
*   下記のARTICLE_TO_SUMMARIZE =""の中身を自由に変更して要約を実行
*   今回は例文として[こちら](https://www3.nhk.or.jp/news/html/20201020/k10012671361000.html?utm_int=news_contents_news-main_007)のニューステキスト（一部）を用いる



---
上記ニュースサイトから一部抜粋したソーステキスト

世界を代表するフランスの建築家、ル・コルビュジエが90年余り前に改修した船が、日本の建築家らの支援でパリのセーヌ川から引き揚げられ、今後船上ギャラリーとして再生されることになりました。この船は、世界文化遺産に登録された東京の国立西洋美術館などの設計で知られる、ル・コルビュジエが90年余り前に改修したもので、水平な連続窓などみずから提唱した近代建築の要素が盛り込まれフランスの文化財にも指定されています



In [ ]:
ARTICLE_TO_SUMMARIZE="世界を代表するフランスの建築家、ル・コルビュジエが90年余り前に改修した船が、日本の建築家らの支援でパリのセーヌ川から引き揚げられ、今後船上ギャラリーとして再生されることになりました。この船は、世界文化遺産に登録された東京の国立西洋美術館などの設計で知られる、ル・コルビュジエが90年余り前に改修したもので、水平な連続窓などみずから提唱した近代建築の要素が盛り込まれフランスの文化財にも指定されています"
batch = tokenizer.prepare_seq2seq_batch(src_texts=[ARTICLE_TO_SUMMARIZE], src_lang="ja_XX", max_target_length=1024, truncation=True).to('cuda')
print(batch)

{'input_ids': tensor([[     6,   3221,    251,   6959,   1786, 155480,    154,  31412,   1433,
             37,   5283,   1925,   9368,   5283,  20891,  55545,  13523,  11264,
            281,   5039,    470,  11152,   2694,  20362,  10623,   7981,   2419,
          15065,    281,     37,  44554,  31412,   1433,   4182,    154,  23795,
            507,  21348,   5677,    154,  26221,   4725, 182537,  13399,   1309,
          59917,  50464,  39187,  31296,     37,  71312,  15065,    575, 224801,
           8697,  21711,   5750,  85665,   9576, 212914,     30,   3619,  15065,
            342,     37,   3221,   4278, 224453, 160450,   8346,  22888,    154,
         186026, 182364, 170853,  18009,   6156,    507,   5085,  14446,     37,
           5283,   1925,   9368,   5283,  20891,  55545,  13523,  11264,    281,
           5039,    470,  11152,   2694,  20362,  10623,   7981, 128647,    507,
             37,  13368,   1308, 181797, 182944,   5546,   6095,  21862,   1309,
           974

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


## 要約の生成

*   num_beamsを変えることで，どの範囲まで探索するかを変えることができる（この値を大きくするほど推論は遅くなる）
*   max_lengthで要約の出力最大長を指定する．この長さを超える場合は強制的に終了する


---
生成される要約テキスト（上記サンプルの場合）

ル・コルビュジエが90年余り前に改修した船が、日本の建築家らの支援で引き揚げられた。水平な連続窓など、みずから提唱した近代建築の要素が盛り込まれている。今後船上ギャラリーとして再生されるとのこと。


In [ ]:
translated_tokens = model.generate(**batch, decoder_start_token_id=tokenizer.lang_code_to_id["ja_XX"], max_length=100, early_stopping=True, num_beams=4)
translation = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
print(translation)

ル・コルビュジエが90年余り前に改修した船が、日本の建築家らの支援で引き揚げられた。水平な連続窓など、みずから提唱した近代建築の要素が盛り込まれている。今後船上ギャラリーとして再生されるとのこと。
